# Aula 006 - Memória de Conversação com RunnableWithMessageHistory

Este notebook demonstra como adicionar memória de conversação a uma cadeia LCEL
usando RunnableWithMessageHistory, permitindo que o modelo mantenha contexto
entre múltiplas interações.

**Conceitos abordados:**
- ChatPromptTemplate com placeholder para histórico
- InMemoryChatMessageHistory: Armazenamento de mensagens em memória
- RunnableWithMessageHistory: Wrapper que adiciona memória a cadeias
- Gerenciamento de sessões para múltiplas conversas
- Operador pipe (|): Sintaxe LCEL para encadeamento

**Caso de uso:**
- Chatbot de recomendação de cidades turísticas
- Conversa multi-turno com contexto preservado

**Fluxo da cadeia:**
```
{query, historico} -> ChatPromptTemplate -> ChatOpenAI -> StrOutputParser -> string
                           ↑
             RunnableWithMessageHistory (gerencia histórico por sessão)
```

**Dependências:**
- langchain-openai: Integração do LangChain com OpenAI
- langchain-core: Output parsers, histórico de mensagens
- python-dotenv: Gerenciamento de variáveis de ambiente

## Importações e Configuração

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.globals import set_debug

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtém a chave da API
api_key = os.getenv('OPENAI_API_KEY')

## Criação do Modelo e do Prompt com Histórico

O placeholder `{historico}` será automaticamente preenchido com as mensagens anteriores da conversa.

In [ ]:
# Inicialização do modelo
modelo = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=api_key,
    temperature=0.7,
    max_tokens=500
)

# Criação do prompt com suporte a histórico de conversa
prompt_sugestao = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um assistente especializado em recomendar cidades turísticas com base nos interesses do usuário."),
        ("placeholder", "{historico}"),
        ("human", "{query}"),
    ]
)

# Criação da cadeia base
cadeia = prompt_sugestao | modelo | StrOutputParser()

## Configuração da Memória de Conversação

- `memoria`: Dicionário que armazena o histórico por sessão
- `historico_por_sessao`: Função que retorna o histórico de uma sessão específica
- `RunnableWithMessageHistory`: Wrapper que gerencia automaticamente o histórico

In [ ]:
# Dicionário para armazenar o histórico de cada sessão
memoria = {}
sessao_id = "aula006_sessao1"

def historico_por_sessao(sessao_id: str) -> InMemoryChatMessageHistory:
    """Retorna o histórico de mensagens para uma sessão específica."""
    if sessao_id not in memoria:
        memoria[sessao_id] = InMemoryChatMessageHistory()
    return memoria[sessao_id]

# Cria a cadeia com memória
cadeia_com_memoria = RunnableWithMessageHistory(
    runnable=cadeia,
    get_session_history=historico_por_sessao,
    input_messages_key="query",
    history_messages_key="historico",
)

## Conversa Multi-turno

Observe como a segunda pergunta faz referência à primeira resposta, demonstrando que o modelo mantém o contexto da conversa.

In [ ]:
# Lista de perguntas para simular uma conversa multi-turno
lista_pergunta = [
    "Quero visitar cidades com muitas praias. Quais você recomenda?",
    "Qual a melhor epoca do ano para visitar essas cidades?",
]

for pergunta in lista_pergunta:
    resposta = cadeia_com_memoria.invoke(
        {
            "query": pergunta
        },
        config={"session_id": sessao_id}
    )

    print("Usuario: ", pergunta)
    print("Assistente: ", resposta, "\n")